In [1]:
import pandas as pd
import sqlite3

# File paths
file_paths = {
    "lot_activity": r"C:\Users\13142\Downloads\1_lot_activity.csv",
    "dim_community": r"C:\Users\13142\Downloads\dim_community.csv",
    "building_elevation_avail": r"C:\Users\13142\Downloads\2_building_elevation_avail.csv",
    "dim_building": r"C:\Users\13142\Downloads\dim_building.csv",
    "lot_construction_activities": r"C:\Users\13142\Downloads\3_lot_construction_activities.csv",
    "community_trans": r"C:\Users\13142\Downloads\4_community_trans.csv",
    "community_budget": r"C:\Users\13142\Downloads\4_community_budget.csv",
    "dim_smh_workers": r"C:\Users\13142\Downloads\dim_smh_workers.csv",
    "dim_construction_activity": r"C:\Users\13142\Downloads\dim_construction_activity.csv",
}

# Load data into Pandas DataFrames
dataframes = {name: pd.read_csv(path) for name, path in file_paths.items()}

# Create SQLite in-memory database
conn = sqlite3.connect(":memory:")

# Load data into SQLite tables
for name, df in dataframes.items():
    df.to_sql(name, conn, index=False, if_exists="replace")

# Question 1: Extract Community Status
query_1 = """
WITH CommunityMetrics AS (
    SELECT
        la.COMM_CODE,
        COUNT(DISTINCT CASE WHEN la.EVENT_VALUE = 'REL' THEN la.LOT_ID END) AS ReleasedLots,
        COUNT(DISTINCT CASE WHEN la.EVENT_VALUE = 'CON' THEN la.LOT_ID END) - 
        COUNT(DISTINCT CASE WHEN la.EVENT_VALUE IN ('CAN', 'TRA') THEN la.LOT_ID END) AS NetSales,
        COUNT(DISTINCT CASE WHEN la.EVENT_TYPE = 6 THEN la.LOT_ID END) AS Closings
    FROM lot_activity la
    GROUP BY la.COMM_CODE
)
SELECT
    cm.COMM_CODE,
    dc.COMM_NAME,
    dc.STATE,
    cm.ReleasedLots,
    cm.NetSales,
    cm.Closings,
    CASE
        WHEN cm.ReleasedLots = 0 THEN 'Future'
        WHEN cm.ReleasedLots = cm.NetSales THEN 'Sold Out'
        WHEN cm.ReleasedLots = cm.Closings THEN 'Closed Out'
        ELSE 'Open For Sale'
    END AS CommunityStatus
FROM CommunityMetrics cm
JOIN dim_community dc ON cm.COMM_CODE = dc.COMM_CODE
ORDER BY CommunityStatus, cm.COMM_CODE;
"""
result_1 = pd.read_sql_query(query_1, conn)
print("Question 1 Output:")
print(result_1.head())
result_1.to_csv(r"C:\Users\13142\Downloads\question_1_output.csv", index=False)

# Question 2: Transpose Elevations by Community-Building
query_2 = """
SELECT 
    elev.COMM_CODE AS Community, 
    dim_bldg.BUILD_NAME AS Building,
    MAX(CASE WHEN elev.STATUS = 'Pending Release-Bidding (PR)' THEN elev.ELEVATION ELSE NULL END) AS "Pending Release-Bidding (PR)",
    MAX(CASE WHEN elev.STATUS = 'Release to Permit (RP)' THEN elev.ELEVATION ELSE NULL END) AS "Release to Permit (RP)",
    MAX(CASE WHEN elev.STATUS = 'Release to Sales (RS)' THEN elev.ELEVATION ELSE NULL END) AS "Release to Sales (RS)",
    MAX(CASE WHEN elev.STATUS = 'Closed Out (CO)' THEN elev.ELEVATION ELSE NULL END) AS "Closed Out (CO)",
    MAX(CASE WHEN elev.STATUS = 'Discontinue Sales (DS)' THEN elev.ELEVATION ELSE NULL END) AS "Discontinue Sales (DS)"
FROM building_elevation_avail elev
JOIN dim_building dim_bldg  
   ON elev.BUILD_CODE = dim_bldg.BUILD_CODE
GROUP BY elev.COMM_CODE, dim_bldg.BUILD_NAME;

"""
result_2 = pd.read_sql_query(query_2, conn)
print("Question 2 Output:")
print(result_2.head())
result_2.to_csv(r"C:\Users\13142\Downloads\question_2_output.csv", index=False)

# Question 3: Last and Next Construction Activities by Lot
query_3 = """
SELECT 
    dca.CONSTRUCTION_ACTIVITY_NAME,
    MAX(CASE WHEN lca.ACTIVITY_CLOSED = 1 THEN dca.CONSTRUCTION_ACTIVITY_NAME END) AS last_completed_activity,
    MAX(CASE WHEN lca.ACTIVITY_CLOSED = 1 THEN dsw.SMH_WORKER_NAME END) AS last_completed_worker,
    MIN(CASE WHEN lca.ACTIVITY_CLOSED = 0 THEN dca.CONSTRUCTION_ACTIVITY_NAME END) AS next_completed_activity,
    MIN(CASE WHEN lca.ACTIVITY_CLOSED = 0 THEN dsw.SMH_WORKER_NAME END) AS next_completed_worker
FROM lot_construction_activities lca
JOIN dim_construction_activity dca ON lca.CONSTRUCTION_ACTIVITY_ID = dca.CONSTRUCTION_ACTIVITY_ID
JOIN dim_smh_workers dsw ON lca.SMH_WORKER_ID = dsw.SMH_WORKER_ID
GROUP BY dca.CONSTRUCTION_ACTIVITY_NAME
ORDER BY MIN(lca.CONSTRUCTION_ACTIVITY_ID)-- Use this trick to sort by CONSTRUCTION_ACTIVITY_ID without including it directly in the GROUP BY clause;
"""
result_3 = pd.read_sql_query(query_3, conn)
print("Question 3 Output:")
print(result_3.head())
result_3.to_csv(r"C:\Users\13142\Downloads\question_3_output.csv", index=False)

# Question 4: Spend Performance by Community-Cost Category
query_4 = """
SELECT 
    ct.COMM_CODE,
    ct.TRANS_DATE,
    dc.COMM_NAME,
    dc.DIV_NAME,
    dc.COUNTY,
    dc.STATE,
    ct.COST_CATEGORY_CODE,
    SUM(ct.AMOUNT) AS Actual_Spend,
    SUM(cb.BUDGET) AS Budget,
    SUM(ct.AMOUNT) - SUM(cb.BUDGET) AS Spend_performance
FROM 
    community_trans ct
LEFT JOIN 
    community_budget cb 
ON 
    ct.COMM_CODE = cb.COMM_CODE 
    AND ct.COST_CATEGORY_CODE = cb.COST_CATEGORY_CODE
LEFT JOIN 
    dim_community dc 
ON 
    ct.COMM_CODE = dc.COMM_CODE
GROUP BY 
    ct.COMM_CODE, 
    dc.COMM_NAME
   ;

"""
result_4 = pd.read_sql_query(query_4, conn)
print("Question 4 Output:")
print(result_4.head())
result_4.to_csv(r"C:\Users\13142\Downloads\question_4_output.csv", index=False)

# Close the SQLite connection
conn.close()


Question 1 Output:
  COMM_CODE         COMM_NAME STATE  ReleasedLots  NetSales  Closings  \
0      SMH1   SMH Community 1    CA           352       318       352   
1     SMH23  SMH Community 23    CA            14        13        14   
2      SMH3   SMH Community 3    CA            72        70        72   
3      SMH8   SMH Community 8    NM            69        60        69   
4      SMH9   SMH Community 9    CA            58        51        58   

  CommunityStatus  
0      Closed Out  
1      Closed Out  
2      Closed Out  
3      Closed Out  
4      Closed Out  
Question 2 Output:
  Community     Building Pending Release-Bidding (PR) Release to Permit (RP)  \
0      SMH1   Building 1                            G                   None   
1      SMH1  Building 10                            P                   None   
2      SMH1   Building 2                           G4                   None   
3      SMH1  Building 26                         None                   None   
4  